In [2]:
import pickle
import json
import os
import re
import sys
from copy import deepcopy
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import skimage
import tifffile
import yaml
from matplotlib.collections import PolyCollection
from matplotlib.colors import TwoSlopeNorm
from einops import rearrange, repeat
from pydantic_extra_types.color import Color

In [3]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'

In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

In [6]:
from mushroom.mushroom import Mushroom, DEFAULT_CONFIG
import mushroom.utils as utils
import mushroom.visualization.utils as vis_utils
import mushroom.data.multiplex as multiplex
import mushroom.data.visium as visium
import mushroom.data.xenium as xenium
import mushroom.data.cosmx as cosmx
import mushroom.visualization.tiling_utils as tiling_utils

In [7]:
source_root = '/diskmnt/Projects/Users/estorrs/mushroom/data'
target_root = '/data/estorrs/mushroom/data'

In [8]:
run_dir = '/data/estorrs/mushroom/data/projects/submission_v1'

In [9]:
def alter_filesystem(config, source_root, target_root):
    for entry in config['sections']:
        for mapping in entry['data']:
            mapping['filepath'] = mapping['filepath'].replace(source_root, target_root)
        
    return config

In [10]:
disease_to_cases = {
    'brca': ['HT704B1', 'HT206B1', 'HT397B1'],
    'prad': ['HT891Z1', 'HT913Z1']
}

case_to_dir = {
    k:Path(f'{run_dir}/{k}') for d in disease_to_cases.values() for k in d
}
case_to_config = {
    k:alter_filesystem(
        yaml.safe_load(open(os.path.join(v, 'registered', 'metadata.yaml'))), source_root, target_root)
    for k, v in case_to_dir.items()
}

In [11]:
def get_roi_to_data(result_dir, region_dir):
    fps = sorted(utils.listfiles(result_dir))

    roi_to_results = {}
    for fp in fps:
        name = fp.split('/')[-1].split('.')[0]
        roi = re.sub(r'^(roi[0-9]+([^_]*)?)_(.*_.*)_(.*)$', r'\1', name)
        dti = re.sub(r'^(roi[0-9]+([^_]*)?)_(.*_.*)_(.*)$', r'\3', name)
        file_type = re.sub(r'^(roi[0-9]+([^_]*)?)_(.*_.*)_(.*)$', r'\4', name)
    
        if roi not in roi_to_results:
            roi_to_results[roi] = {}
        if dti not in roi_to_results[roi]:
            roi_to_results[roi][dti] = {}
    
        if file_type == 'metadata':
            roi_to_results[roi][dti][file_type] = json.load(open(fp))
        else:
            roi_to_results[roi][dti][file_type] = pd.read_csv(fp, sep='\t', index_col=0)

    
    roi_to_data = {}
    for roi, results in roi_to_results.items():
        fp = region_dir / f'{roi}_metadata.json'
        data = json.load(open(fp))
        data['results'] = results
        roi_to_data[roi] = data
    

    return roi_to_data

In [12]:
dti_to_color = {
    'xenium_0': sns.color_palette('deep')[0],
    'xenium_1': sns.color_palette('deep')[0],
    'xenium_combined': sns.color_palette('deep')[0],
    'codex_0': sns.color_palette('deep')[2],
    'codex_1': sns.color_palette('deep')[2],
    'cosmx_0': sns.color_palette('deep')[5],
    'vishd_0': sns.color_palette('deep')[8],
    'visium_0': sns.color_palette('deep')[3],
}